In [1]:
import os
import sys
import numpy as np
from matchms.importing import load_from_json
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

ROOT = os.path.dirname(os.getcwd())
sys.path.insert(0, ROOT)
path_data = 'C:\\OneDrive - Netherlands eScience Center\\Project_Wageningen_iOMEGA\\matchms\\data\\'

## Load spectra

In [2]:
filename = os.path.join(path_data,'gnps_positive_ionmode_cleaned_by_matchms_and_lookups.json')
spectrums = load_from_json(filename)

print("number of spectra:", len(spectrums))

number of spectra: 112956


In [3]:
import pickle
pickle.dump(spectrums, 
            open(os.path.join(path_data,'gnps_positive_ionmode_cleaned_by_matchms_and_lookups.pickle'), "wb"))

In [ ]:
import pickle
outfile = os.path.join(path_data,'gnps_positive_ionmode_cleaned_by_matchms_and_lookups.pickle')
with open(outfile, 'rb') as file:
    spectrums = pickle.load(file)

In [4]:
from matchms.filtering import normalize_intensities
from matchms.filtering import require_minimum_number_of_peaks
from matchms.filtering import select_by_mz
from matchms.filtering import select_by_relative_intensity
from matchms.filtering import reduce_to_number_of_peaks
from matchms.filtering import add_losses

In [5]:
def post_process(s):
    s = normalize_intensities(s)
    s = select_by_mz(s, mz_from=10.0, mz_to=1000)
    s = require_minimum_number_of_peaks(s, n_required=10)
    return s

# apply post processing steps to the data
spectrums = [post_process(s) for s in spectrums]

# omit spectrums that didn't qualify for analysis
spectrums = [s for s in spectrums if s is not None]

print("Number of remaining spectra:", len(spectrums))

Number of remaining spectra: 95319


In [6]:
import pickle
pickle.dump(spectrums, 
            open(os.path.join(path_data,'gnps_positive_ionmode_processed.pickle'), "wb"))

In [6]:
import pickle
outfile = os.path.join(path_data,'gnps_positive_ionmode_processed.pickle')
with open(outfile, 'rb') as file:
    spectrums = pickle.load(file)

### Minimum filtering

In [7]:
number_of_peaks = [len(spec.peaks) for spec in spectrums]

print("Maximum number of peaks in one spectrum:", np.max(number_of_peaks))
print("Number of spectra with > 1000 peaks:", np.sum(np.array(number_of_peaks)>1000))
print("Number of spectra with > 2000 peaks:", np.sum(np.array(number_of_peaks)>2000))
print("Number of spectra with > 5000 peaks:", np.sum(np.array(number_of_peaks)>5000))
print("Careful: Number of spectra with < 10 peaks:", np.sum(np.array(number_of_peaks)<10))

Maximum number of peaks in one spectrum: 228989
Number of spectra with > 1000 peaks: 5481
Number of spectra with > 2000 peaks: 2268
Number of spectra with > 5000 peaks: 769
Careful: Number of spectra with < 10 peaks: 0


In [27]:
ID = 102
if spectrums[ID].get("inchi") + spectrums[ID].get("smiles"):
    print(spectrums[ID].get("inchi") + "\n\n" + spectrums[ID].get("smiles"))

InChI=1S/C28H26N4O3/c1-28-26(34-3)17(29-2)12-20(35-28)31-18-10-6-4-8-14(18)22-23-16(13-30-27(23)33)21-15-9-5-7-11-19(15)32(28)25(21)24(22)31/h4-11,17,20,26,29H,12-13H2,1-3H3,(H,30,33)

CNC1CC2OC(C)(C1OC)N1C3=CC=CC=C3C3=C4CNC(=O)C4=C4C5=C(C=CC=C5)N2C4=C13


In [40]:
annotation_list = []
annotation_list2 = []
for i, s in enumerate(spectrums):
    if s.get("inchi") or s.get("smiles"):
        annotation_list.append((i, s.get("inchi"), s.get("smiles")))
        
    if s.get("inchikey"):
        annotation_list2.append((i, s.get("inchikey")))
        

In [41]:
len(annotation_list), len(annotation_list2)

(77144, 77091)

In [50]:
#spectrums_annotated = [s for s in spectrums if s.get("inchi") or s.get("smiles")]
spectrums_annotated = [s for s in spectrums if s.get("inchikey")]

In [51]:
len(spectrums_annotated), len(spectrums)

(77091, 95319)

## Define functions to make spectrum vector for ML model

In [2]:
def bin_size_fixed(number, d_bins):
    return d_bins * number

def bin_number_fixed(mz, d_bins):
    """Return binned position"""
    return int(mz/d_bins)

def bin_number_array_fixed(mz_array, d_bins, mz_min):
    """Return binned position"""
    assert np.all(mz_array >= mz_min), "Found peaks > mz_min."
    bins = mz_array/d_bins
    return (bins - bin_number_fixed(mz_min, d_bins)).astype(int)

def set_d_bins_fixed(number_of_bins, mz_min=10.0, mz_max=1000.0):
    return (mz_max - mz_min) / number_of_bins

In [3]:
def create_dictionary_fixed(spectrums, d_bins, mz_min):
    unique_peaks = set()
    for spectrum in spectrums:
        for mz in bin_number_array_fixed(spectrum.peaks.mz, d_bins, mz_min):
            unique_peaks.add(mz)
    unique_peaks = sorted(unique_peaks)
    class_values = {}

    for i, item in enumerate(unique_peaks):
        class_values[item] = i
    
    return class_values, unique_peaks

In [4]:
def create_vectors_fixed(spectrums, d_bins, mz_min=10.0, weight_power = 0.2):
    """Create documents."""
    documents = []
    #documents_weights = []
    #documents_bow = []

    for spectrum in spectrums:
        doc = bin_number_array_fixed(spectrum.peaks.mz, d_bins, mz_min=mz_min)
        weights = spectrum.peaks.intensities ** weight_power
        doc_bow = [class_values[x] for x in doc]
        documents.append(list(zip(doc_bow, weights)))
        #documents_weights.append(weights) 
        #documents_bow.append([class_values[x] for x in doc])
    
    return documents

def create_vector(document, vector_dim):
    """Create vector from document."""
    bow, weights = zip(*document)
    vector_base = np.zeros((1, vector_dim))
    vector = (np.array(weights) * to_categorical(np.array(bow)).T).max(axis=1)
    vector_base[0, :vector.shape[0]] += vector
    return vector_base

In [5]:
import numba

@numba.njit
def create_full_vector(document, vector_dim):
    """Create full (sparse) vector from document."""
    full_vector = np.zeros((vector_dim))
    for (ID, weight) in document:
        full_vector[ID] = max(weight, full_vector[ID])
    return full_vector

def create_peak_dict(document):
    peaks = {}
    for (ID, weight) in document:
        if ID in peaks:
            peaks[ID] = max(weight, peaks[ID])
        else:
            peaks[ID] = weight
    return peaks

## Create reference scores (Tanimoto)
- Check better alternatives?

In [87]:
from matchms.filtering.add_fingerprint import add_fingerprint

spectrums_annotated = [add_fingerprint(s, fingerprint_type="daylight", nbits=2048) for s in spectrums_annotated]

In [88]:
import pickle
pickle.dump(spectrums_annotated, 
            open(os.path.join(path_data,'gnps_positive_ionmode_annotated_w_fps_processed.pickle'), "wb"))

In [6]:
import pickle
outfile = os.path.join(path_data,'gnps_positive_ionmode_annotated_w_fps_processed.pickle')
with open(outfile, 'rb') as file:
    spectrums_annotated = pickle.load(file)

In [7]:
inchikeys_list = []
for s in spectrums_annotated:
    inchikeys_list.append(s.get("inchikey"))

inchikeys_array = np.array(inchikeys_list)

In [8]:
inchi_list = []
for s in spectrums_annotated:
    inchi_list.append(s.get("inchi"))

inchi_array = np.array(inchi_list)

In [57]:
inchikeys_unique = list(set(inchikeys_list))
len(inchikeys_unique)

14459

In [10]:
from collections import Counter 
  
def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 

In [58]:
inchikey = inchikeys_unique[1000]

idx = np.where(inchikeys_array == inchikey)[0]
for i in idx:
    print(spectrums_annotated[i].get("smiles") + "\n")

print("most frequent:", most_frequent([spectrums_annotated[i].get("smiles") for i in idx]))

O=C(N1CCC=CC1=O)/C=C/C2=CC(OC)=C(OC)C(OC)=C2

COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC

COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC

COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC

COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC

COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC

COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC

COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC

COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC

COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC

COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC

COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC

COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC

COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC

most frequent: COc1cc(/C=C/C(=O)N2CCC=CC2=O)cc(OC)c1OC


In [59]:
inchi_mapping = []
ID_mapping = []

for inchikey in inchikeys_unique:
    idx = np.where(inchikeys_array == inchikey)[0]
    
    inchi = most_frequent([spectrums_annotated[i].get("inchi") for i in idx])
    inchi_mapping.append(inchi)
    ID = idx[np.where(inchi_array[idx] == inchi)[0][0]]
    ID_mapping.append(ID)

In [63]:
metadata = pd.read_csv("metadata_AllInchikeys_safe.csv")
metadata.head()

,Unnamed: 0,inchikey,inchi,ID
0,0,MYHSVHWQEVDFQT-RLIDIOMENA-N,InChI=1/C11H19NO10S2/c1-2-5(14)3-7(12-22-24(18...,75971
1,1,BKAWJIRCKVUVED-UHFFFAOYSA-N,"InChI=1S/C6H9NOS/c1-5-6(2-3-8)9-4-7-5/h4,8H,2-...",17330
2,2,CXVGEDCSTKKODG-UHFFFAOYSA-N,InChI=1S/C14H12O6S/c1-20-12-8-11(15)10(7-13(12...,422
3,3,JAMSDVDUWQNQFZ-QNQJCTKXSA-N,InChI=1S/C52H102NO8P/c1-6-8-10-12-14-16-18-20-...,38937
4,4,ODHCTXKNWHHXJC-GSVOUGTGSA-N,"InChI=1S/C5H7NO3/c7-4-2-1-3(6-4)5(8)9/h3H,1-2H...",46378


In [66]:
metadata["inchikey"].values.shape

(14459,)

In [60]:
import pandas as pd
metadata = pd.DataFrame(list(zip(inchikeys_unique, inchi_mapping, ID_mapping)), columns=["inchikey", "inchi", "ID"])
metadata.to_csv("metadata_AllInchikeys.csv")

metadata.head()

,inchikey,inchi,ID
0,MYHSVHWQEVDFQT-RLIDIOMENA-N,InChI=1/C11H19NO10S2/c1-2-5(14)3-7(12-22-24(18...,75971
1,BKAWJIRCKVUVED-UHFFFAOYSA-N,"InChI=1S/C6H9NOS/c1-5-6(2-3-8)9-4-7-5/h4,8H,2-...",17330
2,CXVGEDCSTKKODG-UHFFFAOYSA-N,InChI=1S/C14H12O6S/c1-20-12-8-11(15)10(7-13(12...,422
3,JAMSDVDUWQNQFZ-QNQJCTKXSA-N,InChI=1S/C52H102NO8P/c1-6-8-10-12-14-16-18-20-...,38937
4,ODHCTXKNWHHXJC-GSVOUGTGSA-N,"InChI=1S/C5H7NO3/c7-4-2-1-3(6-4)5(8)9/h3H,1-2H...",46378


In [16]:
#check
spectrums_annotated[75971].get("inchi"), inchikeys_unique[0], len(metadata)

('InChI=1/C11H19NO10S2/c1-2-5(14)3-7(12-22-24(18,19)20)23-11-10(17)9(16)8(15)6(4-13)21-11/h2,5-6,8-11,13-17H,1,3-4H2,(H,18,19,20)/b12-7+/t5-,6+,8+,9-,10+,11u/m0/s1/f/h18H',
 'MYHSVHWQEVDFQT-RLIDIOMENA-N',
 14459)

In [15]:
metadata.ID.values.shape

(14459,)

In [67]:
from matchms.similarity import FingerprintSimilarity

spectrums_represent = [spectrums_annotated[i] for i in metadata.ID.values]

similarity_measure = FingerprintSimilarity(similarity_measure="jaccard")
scores_mol_similarity = similarity_measure.matrix(spectrums_represent, spectrums_represent)

filename = os.path.join(path_data, "similarities_AllInchikeys_daylight2048_jaccard.npy")
np.save(filename, scores_mol_similarity)

### Load reference scores (Tanimoto)

In [17]:
filename = os.path.join(path_data,'similarities_AllInchikeys_daylight2048_jaccard.npy')
scores_mol_similarity = np.load(filename)

In [18]:
scores_mol_similarity.shape

(14459, 14459)

In [68]:
scores_mol_similarity[:10,:5]

array([[1.        , 0.09930716, 0.23619371, 0.20579111, 0.09101251],
       [0.09930716, 1.        , 0.09296782, 0.08462867, 0.06852792],
       [0.23619371, 0.09296782, 1.        , 0.14127144, 0.1026253 ],
       [0.20579111, 0.08462867, 0.14127144, 1.        , 0.13392857],
       [0.09101251, 0.06852792, 0.1026253 , 0.13392857, 1.        ],
       [0.17615176, 0.07909605, 0.19981061, 0.11771429, 0.08438819],
       [0.29403307, 0.10798946, 0.26704953, 0.20884521, 0.1674333 ],
       [0.21709402, 0.10336239, 0.20993031, 0.16033755, 0.18435013],
       [0.18895966, 0.07380074, 0.13493724, 0.18452381, 0.09683426],
       [0.11052072, 0.08798283, 0.18963486, 0.08912387, 0.12227074]])

In [81]:
ID = 100
np.where((scores_mol_similarity[ID,:] > 0.7) & (scores_mol_similarity[ID,:] < 0.99))[0]

array([ 3339,  5594, 10540], dtype=int64)

In [79]:
metadata.loc[ID]["inchi"], metadata.loc[3339]["inchi"]

('InChI=1S/C20H39NO4S/c1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-20(22)21-18-19-26(23,24)25/h9-10H,2-8,11-19H2,1H3,(H,21,22)(H,23,24,25)/b10-9-',
 'InChI=1S/C21H41NO4S/c1-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-21(23)22(2)19-20-27(24,25)26/h10-11H,3-9,12-20H2,1-2H3,(H,24,25,26)/b11-10-')

## Build data pipeline for training on AllInchikeys dataset

In [90]:
split_ratio = (0.85, 0.075, 0.075)

np.random.seed(111) #42

N_label = len(inchikeys_unique)
idx = np.arange(0, N_label)
N_train = int(split_ratio[0] * N_label)
N_val = int(split_ratio[1] * N_label)
N_test = N_label - N_train - N_val
print("Split dataset into train/val/test fractions:", N_train, N_val, N_test)

# Select training, validation, and test IDs:
trainIDs = np.random.choice(idx, N_train, replace=False)
valIDs = np.random.choice(list(set(idx) - set(trainIDs)), N_val, replace=False)
testIDs = list(set(idx) - set(trainIDs) - set(valIDs))

Split dataset into train/val/test fractions: 12290 1084 1085


In [91]:
trainIDs[:10]

array([11925,  2413, 13459, 12783, 13292,  1785, 13828,   637,  7179,
       10400])

## Convert spectra into binned spectra and peak dicts

0.1

In [23]:
# Set binning --> LINEAR
number_of_bins = 10000
mz_max = 1000.0
mz_min = 10.0
weight_power = 0.5

d_bins = set_d_bins_fixed(number_of_bins, mz_min=mz_min, mz_max=mz_max)
print("d_bins:", d_bins)

class_values, unique_peaks = create_dictionary_fixed(spectrums_annotated, d_bins, mz_min)
vector_dim = len(unique_peaks)
print("Vector dimension:", vector_dim)

d_bins: 0.099
Vector dimension: 9998


In [25]:
spectrums_binned = create_vectors_fixed(spectrums_annotated, d_bins, mz_min=mz_min, weight_power=weight_power)
spectrums_binned_dicts = [create_peak_dict(spec) for spec in spectrums_binned]

In [84]:
len(spectrums_binned_dicts)

77091

## Try more balanced training data

In [103]:
from tensorflow.keras.utils import Sequence

class DataGeneratorAllInchikey(Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=32, num_turns=1, dim=(10000,1), n_channels=1,
                 n_classes=1, shuffle=True, score_array=None, inchikey_list= None, inchikey_mapping=None, 
                 same_prob_bins=[(0, 0.5), (0.5, 1)],
                augment_peak_removal={"max_removal": 0.2, "max_intensity": 0.2},
                 augment_intensity=0.1):
        """Initialization"""
        self.dim = dim
        self.batch_size = batch_size
        self.num_turns = num_turns #number of go's through all IDs
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        assert score_array is not None, "needs score array"
        self.score_array = score_array
        assert inchikey_mapping is not None, "needs inchikey mapping"
        self.inchikey_mapping = inchikey_mapping
        self.inchikey_list = inchikey_list
        self.score_array[np.isnan(score_array)] = 0
        self.same_prob_bins = same_prob_bins
        self.augment_peak_removal = augment_peak_removal
        self.augment_intensity = augment_intensity

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(self.num_turns) * int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data"""
        # Go through all indexes
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Select subset of IDs
        list_IDs_temp = []
        for index in indexes:
            ID1 = self.list_IDs[index]
            prob_bins = self.same_prob_bins[np.random.choice(np.arange(len(self.same_prob_bins)))]
            
            idx = np.where((self.score_array[ID1, self.list_IDs] > prob_bins[0]) 
                           & (self.score_array[ID1, self.list_IDs] <= prob_bins[1]))[0]
            if len(idx) > 0:
                ID2 = self.list_IDs[np.random.choice(idx)]
            else:
                ID2 = self.list_IDs[np.random.randint(0, len(self.list_IDs))]
                    
            list_IDs_temp.append((ID1, ID2))

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        self.indexes = np.tile(np.arange(len(self.list_IDs)), int(self.num_turns))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_augmentation(self, document_dict):
        """Data augmentation."""
        idx = np.array([x for x in document_dict.keys()])
        values = np.array([x for x in document_dict.values()])
        if self.augment_peak_removal:
            indices_select = np.where(values < self.augment_peak_removal["max_intensity"])[0]
            removal_part = np.random.random(1) * self.augment_peak_removal["max_removal"]
            indices_select = np.random.choice(indices_select,
                                              int(np.ceil((1 - removal_part)*len(indices_select))))
            indices = np.concatenate((indices_select,
                                      np.where(values >= self.augment_peak_removal["max_intensity"])[0]))
            if len(indices) > 0:
                idx = idx[indices]
                values = values[indices]
        if self.augment_intensity:
            values = (1 - self.augment_intensity * 2 * (np.random.random(values.shape) - 0.5)) * values
        return idx, values
            
            
    def __data_generation(self, list_IDs_temp):
        """Generates data containing batch_size samples""" # X : (n_samples, *dim, n_channels)
        # Initialization
        X = [np.zeros((self.batch_size, 1, self.dim)) for i in range(2)]
        y = np.zeros((self.batch_size,))
        
        # Generate data
        for i, IDs in enumerate(list_IDs_temp):
            # Select spectrum for respective inchikey
            inchikey_1 = self.inchikey_mapping.loc[IDs[0]]["inchikey"]
            inchikey_2 = self.inchikey_mapping.loc[IDs[1]]["inchikey"]
            ID1 = np.random.choice(np.where(self.inchikey_list == inchikey_1)[0])
            ID2 = np.random.choice(np.where(self.inchikey_list == inchikey_2)[0])
            
            ref_idx, ref_values = self.__data_augmentation(spectrums_binned_dicts[ID1])
            X[0][i, 0, ref_idx] = ref_values
            query_idx, query_values = self.__data_augmentation(spectrums_binned_dicts[ID2])
            X[1][i, 0, query_idx] = query_values
            y[i] = self.score_array[IDs[0], IDs[1]]
            if np.isnan(y[i]):
                y[i] = np.random.random(1)
        return X, y

In [104]:
same_prob_bins = list(zip(0.1*np.arange(10), (0.1 + 0.1*np.arange(10))))

train_generator = DataGeneratorAllInchikey(trainIDs, batch_size=16, num_turns=2, dim=(vector_dim), n_channels=1,
                                           n_classes=1, shuffle=True,
                                           score_array=scores_mol_similarity, 
                                           inchikey_mapping=metadata, inchikey_list = inchikeys_array,
                                           same_prob_bins=same_prob_bins,
                                           augment_peak_removal={"max_removal": 0.2, "max_intensity": 0.1**0.5},
                                           augment_intensity=0.2)

val_generator = DataGeneratorAllInchikey(valIDs, batch_size=16, num_turns=1, dim=(vector_dim), n_channels=1,
                                         n_classes=1, shuffle=True,
                                         score_array=scores_mol_similarity, 
                                         inchikey_mapping=metadata, inchikey_list = inchikeys_array,
                                         same_prob_bins=same_prob_bins,
                                         augment_peak_removal={"max_removal": 0.1, "max_intensity": 0.1**0.5},
                                         augment_intensity=0.1)

In [95]:
A, B = train_generator.__getitem__(0)

In [96]:
A[0].shape, A[1].shape, B

((16, 1, 9998),
 (16, 1, 9998),
 array([0.09530583, 0.18057554, 0.27840482, 0.40562249, 0.56427504,
        0.19261745, 0.51131601, 0.1108871 , 0.16138125, 1.        ,
        0.11970075, 0.31947332, 0.22798295, 0.88427673, 0.81027254,
        0.20757825]))